In [1]:
%reload_ext Cython

In [2]:
import typing
import copy
import pdb
from IPython.display import clear_output
import numpy as np
import time
from colorama import Fore
import sys
import cProfile
from multiprocessing import Pool
import chess 
import boardlib
import chess.uci

In [3]:
all_squares = [[i,j] for i in range(8) for j in range(8)]
all_moves = [i+j for i in all_squares for j in all_squares]
pieces = ["Empty","Rook","Knight","Bishop","Queen","King","Pawn","Pawn","King","Queen","Bishop","Knight","Rook"]

piece_owner = [0,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1]
list_letters = ['a','b','c','d','e','f','g','h']
list_numbers = list(map(str,list((range(1,9)))))
legal_outputs = ["pawn_move","pawn_move,en-passant","king_move",'king_move,castling,kingside','king_move,castling,queenside','en-passant','']
en_passant = ["c7c5","a2a3","c5c4","b2b4","c4b3"]
zero_line = [0 for _ in range(8)]
move_zero_arr = [0 for i in range(8*8)]
flattened_zero_arr = np.zeros((8,8))
flattened_board = np.zeros((8,8,7),dtype=int)

In [4]:
#%%cython
#Creates a new board
def create_board(): 
    #pdb.set_trace()
    val = 0
    board = np.array([[val for j in range(8)] for i in range(8)])
    for i in range(8):
        board[1,i] = 6
        board[6,i] = -6
        
    board[0,4] = 5
    board[7,4] =  -5

    board[0,1] = 2
    board[7,1] = -2

    board[0,6] = 2
    board[7,6] = -2

    board[0,2] = 3
    board[7,2] = -3

    board[0,5] = 3 
    board[7,5] = -3

    board[0,3] = 4
    board[7,3] = -4

    board[0,4] = 5
    board[7,4] = -5
    
    board[0,0] = 1
    board[7,0] = -1 
    
    board[0,7] = 1
    board[7,7] = -1
    return [board]

def create_rights():
    rights = np.array([[0 for j in range(8)] for i in range(8)])
    rights[0,4] = 1
    rights[7,4] = -1
    
    rights[0,0] = 1
    rights[7,0] = -1
    rights[0,7] = 1
    rights[7,7] = -1
    return [rights]

def create_player_positions():
    positions_1 = []
    for i in range(8):
        positions_1.append([0,i])
        positions_1.append([1,i])
    
    positions_2 = []
    for i in range(8):
        positions_2.append([6,i])
        positions_2.append([7,i])
        
    return [[positions_1] + [positions_2]]
    
def create_chess_status():
    return [False]

def create_king_positions():
    king_positions = [[0,4],[7,4]]
    return [king_positions]

def get_from_square_player(i,j,board):
    val = board[i,j]
    return val / abs(val)


def pp_board(board):
    for i in range(8):
        print(Fore.BLACK + list_numbers[7-i],end="\t")
        for j in range(8):
            player = piece_owner[board[i,j]]
            if player == 0:
                print(Fore.BLACK + "_",end="\t")
            else: 
                piece = pieces[board[i,j]]
                if player== 1:
                    print(Fore.BLUE + piece,end="\t")
                else:
                    print(Fore.BLACK + piece,end="\t")
        print("\n")
    print("",end="\t")
    for i in list_letters:
        print(Fore.BLACK + i,end="\t")
    sys.stdout.flush()

In [7]:
class ChessBoard:
    def __init__(self,player):
        self.board = create_board()
        self.rights = create_rights()
        self.king_positions = create_king_positions()
        self.player_positions = create_player_positions()
        self.chess_status = create_chess_status()
        self.player = player
        self.move_status = ""
        self.capture_status = []
    
    def make_move(self,move):
        self.board ,self.move_status ,self.rights ,self.player ,self.king_positions ,self.player_positions ,self.chess_status,self.capture_status = board.make_move(self.board,
                                                                                                                                          move,
                                                                                                                                          self.rights,
                                                                                                                                          self.player,
                                                                                                                                          self.king_positions,
                                                                                                                                          self.player_positions,
                                                                                                                                          self.chess_status)
    def unmake_move(self):
        self.board,self.move_status,self.rights,self.player,self.king_positions,self.player_positions,self.chess_status = board.unmake_move(self.board,
                                                                                                                                            self.rights,
                                                                                                                                            self.player,
                                                                                                                                            self.king_positions,
                                                                                                                                            self.player_positions,
                                                                                                                                            self.chess_status)
    def one_hot_encode_state(self):
        val = self.board[-1]
        one_hot_board = self.one_hot_encode_board(val)
        one_hot_rights_arr = np.asarray(self.rights[-1])
        one_hot_rights_ret = one_hot_rights_arr.flatten()
        return one_hot_board,one_hot_rights_ret
    
    def one_hot_encode_board(self,array):   
        flattened_board_copy = np.zeros((8,8,7),dtype=int) 
        for i in range(8):
            for j in range(8):
                piece_index = int(abs(array[i,j] ))
                sign = boardlib.c_reduce_to_sign(piece_index)
                flattened_board_copy[i,j,int(abs(piece_index))] = sign
        
        return flattened_board_copy.flatten()
    
    def one_hot_encode_move(self,move):
        x_1,y_1,x_2,y_2 = boardlib.algebraic_to_arr_indices(move)
        array_from = copy.deepcopy(move_zero_arr)
        array_to = copy.deepcopy(move_zero_arr)
        index_from = x_1 * 8 + y_1
        index_to = x_2 * 8 + y_2   
        array_from[index_from]  = 1
        array_to[index_to]  = 1   
        return array_from,array_to
 
    def show_board(self):
        pp_board(self.board[-1])
    
    def generate_legal_moves(self):
        return board.generate_all_legal_moves(self.board,self.rights,self.player,
                                        self.king_positions,self.player_positions,
                                        self.chess_status,board.all_moves)
        

In [10]:
b = ChessBoard(1)
np.asarray(b.one_hot_encode_move("a3a2")).reshape(2,8,8)

array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]])

In [ ]:
#%%cython
def curate_pgn_string_from_file(s):
    s = s.replace("\n"," ")
    s = s.replace("+","")
    s = s.replace("Q","")
    s = s.replace("N","")
    s = s.replace("-","")
    s_list = s.split(' ')
    s_list
    list_remove = [s_list[i] for i in range(len(s_list)) if i%3 == 0]
    for i in list_remove:
        s_list.remove(i)
    try:
        s_list.remove("")
    except:
        s_list = s_list
    return s_list

def curate_pgn_string(s):
    f = ["\n","+","Q","N","-","R","x","B","K","#"]
    for i in f:
        s = s.replace(i,"")
    s_list = s.split(' ')
    try:
        s_list.remove("")
    except:
        s_list = s_list
    return s_list
#path = "../extracted/two.txt"
#f = open(path,"r")
#s = f.read()
#curated = curate_pgn_string_from_file(s)

In [ ]:
## Test cell. Here are the functions that test the engine

def perft(b,depth):
    if depth == 0:
        return 1
    nodes = 0
    moves = b.generate_legal_moves()
    
    for i in moves:
        b.make_move(i)
        nodes = nodes + perft(b,depth-1)
        b.unmake_move()
    return nodes

def perft_pool(b,depth):
    if depth == 0:
        return 1
    nodes = 0
    moves = b.generate_legal_moves()
    board_list = []
    for i in moves:
        b.make_move(i)
        board_list.append((copy.deepcopy(b),depth-1))
        #nodes = nodes + perft(b,depth-1)
        b.unmake_move()
    p = Pool()
    retvals = p.starmap_async(perft,board_list)
    #print(retvals.get())
    return sum(retvals.get()) 

def perft_test(depth):
    b = ChessBoard(-1)
    start = time.time()
    val = perft_pool(b,depth)
    end = time.time() 
    return val,end-start

def wrong_moves(correct_board,my_board, depth,path):
    if depth== 0:
        return 1

    nodes = 0
    #pdb.set_trace()
    correct_legal_moves = list(map(correct_board.lan,correct_board.legal_moves))
    my_legal_moves = my_board.generate_legal_moves()
    concated_moves = ""
    for i in correct_legal_moves:
        concated_moves = concated_moves + i + ' '
    correct_legal_moves = curate_pgn_string(concated_moves)
    
    false_positive = ["-" + i for i in my_legal_moves if i not in correct_legal_moves]
    false_negative = ["+" + i for i in correct_legal_moves if i not in my_legal_moves]
    
    correct_legal_moves.sort()
    my_legal_moves.sort()
    if correct_legal_moves != my_legal_moves:
        print("error")
    errors = len(false_positive)+ len(false_negative)
    if errors == 0:
        for i in my_legal_moves:
            path_temp = path + " " + i
            move = chess.Move.from_uci(i)
            
            correct_board.push(move)
            my_board.make_move(i)
            
            nodes = nodes + wrong_moves(correct_board,my_board,depth-1,path_temp)
            
            correct_board.pop()
            my_board.unmake_move()
    else:
        print(path + "\n" + str(false_positive) + str(false_negative) + "\n")
        return 1
    return nodes

In [ ]:
#%%cython
def play_game_until_error(curated_pgn):
    b = ChessBoard(-1)
    count = 0
    for move in curated_pgn:
        #if count == 3:
            #pdb.set_trace()
        b.make_move(move)
        
#        b,s,r,p,k,player_pos,chess= make_move(b,move,r,p,k,player_pos,chess)
        count += 1
        if not(b.move_status in legal_outputs):#== "castling,k" or s == "castling,q" or s == "" or s == "en-passant"):
            print(count,s)
            print(move)
            board.show_board()
            #pp_board(b[-1])
            break
    success = (b.move_status in legal_outputs)# == "castling,k" or s == "castling,q" or s == "" or s == "en-passant")
    return count, move,success,s
#play_game_until_error(curated)

In [ ]:
#%%cython
def start_game(player,moves=[],watching=False,fast_play=False):
    #pdb.set_trace()
    b = ChessBoard(player)
    if fast_play == True:
        for i in moves:
            b.make_move(i)
    count = 0
    do = "m"
    i = 0
    prev = "m"
    stop = False
    b.show_board()
    s = ""
    while not stop:
        do = input()
#        if do == "":
#            do = prev
#            
        if watching:
            if i == len(moves):
                stop = True
                
            if do == "m":
                b.make_move(moves[i])
                prev == "m"
                
                print(moves[i])
            
            elif do == "u":
                b.unmake_move()
                if s == "n":
                    continue
                i = i-2
                prev = "u"
                
                print(moves[i])
            elif do == "exit":
                stop = True
            else:
                print("wrong input")
                continue
        else:  
            if do == "u":
                b.unmake_move()
            
            else:
                b.make_move(do)
                #pdb.set_trace()
                if not(s in legal_outputs):
                    print(s)
                    print(len(s))
                    print("wrong input")
                    sys.stdout.flush()


                    continue

        #clear_output()
        b.show_board()

        sys.stdout.flush()
        count += 1
        i = i + 1
    clear_output()
    b.show_board()

In [ ]:
l = ["a2a4","h7h6","a4a5","h6h5","a5a6","h5h4","a6b7","h4h3","b7a8q","h3g2","a8a7","g2f1"]


#cProfile.run('perft_test(3)')
#perft_test(3)
#wrong_moves(chess.Board(),ChessBoard(-1),4,"")
#perft_2(chess.Board(),3)
#start_game(-1,l,watching=True,fast_play=False)

In [ ]:
def record_stockfish_game():
    board = chess.Board()
    engine = chess.uci.popen_engine("/usr/local/bin/stockfish")
    engine.uci()
    engine.author
    moves = []
    playing = True
    while playing:
        engine.position(board)
        move = engine.go()[0]
        if move == None:
            playing = False
        else:
            board.push(move)
        moves.append(str(move))
    return moves

def save_recording(l,path):
    file = open(path,"a")
    for i in l:
        file.write("%s " % i )
    f.close()

def add_game_to_file():
    moves = record_stockfish_game()
    save_recording(moves,"games.txt")
    
    
def file_to_training_data(path):
    train_x = []
    train_y = []
    file = open(path,"r")
    file_string = file.read()
    file_string = file_string.replace("\n","")
    file_array = file_string.split(" ")
    board = ChessBoard(-1)
    train
    for i in file_array:
        board.make_move(i)
        temp_board = deep.copy(board.self[-1])
        temp_rights = deep.copy(board.rights[-1])
    return file_string

#file_to_training_data("games.txt")
    

In [ ]:
def test_one_hot_encoding():
    b = ChessBoard(-1)
    b_board = b.board[-1]
    b_rights = b.rights[-1]
    one_hot = one_hot_encode_rights(b_rights)
    for i in range(8):
        if i not in [0,1,6,7]:
            continue
        for j in range(8):
            index = i * 8  + j * 7
            index = index + int(abs(b_board[i,j]))
            print(one_hot[index],int(abs(b_board[i,j])))
        